# Data Cleaning


Datapoints that will be used:

Type of mobile (where we want Class A), MMSI, Latitude, Longitude, ROT, SOG, COG, Width, Length, Ship type

If only few datapoints for MMSI, then remove MMSI

In [39]:
import pandas as pd
from pathlib import Path

data_dir = Path("data")
csv_files = sorted(data_dir.glob("*.csv"))

frames = []
for path in csv_files:
    try:
        tmp = pd.read_csv(path)
        tmp["source_file"] = path.name
        frames.append(tmp)
    except Exception as e:
        print(f"Failed to read {path.name}: {e}")

if frames:
    df = pd.concat(frames, ignore_index=True)
else:
    df = pd.DataFrame()

print(f"Loaded {len(frames)} file(s). Total rows: {len(df)}")
df.head()


Failed to read port_locodes.csv: Error tokenizing data. C error: Expected 5 fields in line 4, saw 6

Loaded 2 file(s). Total rows: 21550282


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D,source_file
0,29/03/2024 00:00:00,Class A,219030044,57.028715,9.943790,Unknown value,NaN,0.1,0.0,NaN,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,aisdk-2024-03-29.csv
1,29/03/2024 00:00:00,Class B,219022162,91.000000,0.000000,Unknown value,NaN,NaN,NaN,NaN,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,aisdk-2024-03-29.csv
2,29/03/2024 00:00:00,Base Station,2190068,56.447258,10.945872,Unknown value,NaN,NaN,NaN,NaN,...,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,aisdk-2024-03-29.csv
3,29/03/2024 00:00:00,Class A,219030053,57.058232,9.900808,Unknown value,NaN,0.0,4.0,NaN,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,aisdk-2024-03-29.csv
4,29/03/2024 00:00:00,Class A,219022903,56.152448,10.253387,Reserved for future amendment [HSC],0.0,0.0,201.3,20.0,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,aisdk-2024-03-29.csv


In [40]:
# keep only requested variables
desired_cols = [ "# Timestamp",
    "Type of mobile", "MMSI", "Latitude", "Longitude",
    "ROT", "SOG", "COG", "Width", "Length", "Ship type"
]

available = [c for c in desired_cols if c in df.columns]
missing = [c for c in desired_cols if c not in df.columns]
if missing:
    print(f"Warning: missing columns (will be skipped): {missing}")

df = df[available].copy()
print(f"Selected columns: {available}")
print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
df.head()

Selected columns: ['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'ROT', 'SOG', 'COG', 'Width', 'Length', 'Ship type']
Rows: 21550282, Columns: 11


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,ROT,SOG,COG,Width,Length,Ship type
0,29/03/2024 00:00:00,Class A,219030044,57.028715,9.943790,NaN,0.1,0.0,NaN,NaN,Undefined
1,29/03/2024 00:00:00,Class B,219022162,91.000000,0.000000,NaN,NaN,NaN,NaN,NaN,Undefined
2,29/03/2024 00:00:00,Base Station,2190068,56.447258,10.945872,NaN,NaN,NaN,NaN,NaN,Undefined
3,29/03/2024 00:00:00,Class A,219030053,57.058232,9.900808,NaN,0.0,4.0,NaN,NaN,Undefined
4,29/03/2024 00:00:00,Class A,219022903,56.152448,10.253387,0.0,0.0,201.3,NaN,NaN,Undefined


In [41]:
# count rows with no missing values (no NaN in any column)
mask_no_nan = df.notna().all(axis=1)
n_no_nan = int(mask_no_nan.sum())
total_rows = len(df)
print(f"Rows with no NaN: {n_no_nan} / {total_rows} ({n_no_nan/total_rows:.2%})")

Rows with no NaN: 10406069 / 21550282 (48.29%)


In [42]:
# remove rows that include any NaN
before = len(df)
df = df.dropna().reset_index(drop=True)
after = len(df)
print(f"Removed {before - after} rows containing NaN. Rows now: {after}")
df.head()

Removed 11144213 rows containing NaN. Rows now: 10406069


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,ROT,SOG,COG,Width,Length,Ship type
0,29/03/2024 00:00:01,Class A,255806328,54.721793,12.368540,0.0,10.0,213.9,16.0,107.0,Cargo
1,29/03/2024 00:00:02,Class A,636021210,56.640483,7.389258,5.4,13.1,56.6,32.0,229.0,Cargo
2,29/03/2024 00:00:02,Class A,636021210,56.640483,7.389258,5.4,13.1,56.6,32.0,229.0,Cargo
3,29/03/2024 00:00:03,Class A,220329000,57.122685,8.599583,0.0,0.0,259.8,6.0,18.0,Fishing
4,29/03/2024 00:00:04,Class A,219006113,56.991093,10.308925,0.0,0.0,34.4,4.0,14.0,Fishing


In [43]:
# keep only Class A vessels
df = df[df['Type of mobile'].str.strip() == 'Class A'].reset_index(drop=True)
print(f"Rows after filter (Class A): {len(df)}")
df.head()

Rows after filter (Class A): 10405252


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,ROT,SOG,COG,Width,Length,Ship type
0,29/03/2024 00:00:01,Class A,255806328,54.721793,12.368540,0.0,10.0,213.9,16.0,107.0,Cargo
1,29/03/2024 00:00:02,Class A,636021210,56.640483,7.389258,5.4,13.1,56.6,32.0,229.0,Cargo
2,29/03/2024 00:00:02,Class A,636021210,56.640483,7.389258,5.4,13.1,56.6,32.0,229.0,Cargo
3,29/03/2024 00:00:03,Class A,220329000,57.122685,8.599583,0.0,0.0,259.8,6.0,18.0,Fishing
4,29/03/2024 00:00:04,Class A,219006113,56.991093,10.308925,0.0,0.0,34.4,4.0,14.0,Fishing


In [44]:
# list unique ship types and their counts
ship_type_counts = df['Ship type'].value_counts(dropna=False)
print("Unique ship types:", ship_type_counts.index.tolist())
print("\nCounts:\n", ship_type_counts)

Unique ship types: ['Cargo', 'Fishing', 'Tanker', 'Passenger', 'Other', 'Tug', 'Pilot', 'Dredging', 'HSC', 'SAR', 'Military', 'Undefined', 'Law enforcement', 'Reserved', 'Towing', 'Port tender', 'Sailing', 'Diving', 'Spare 1', 'Pleasure', 'Anti-pollution', 'Not party to conflict', 'Towing long/wide']

Counts:
 Ship type
Cargo                    3095298
Fishing                  1916724
Tanker                   1479575
Passenger                1378058
Other                     470011
Tug                       411055
Pilot                     336473
Dredging                  307014
HSC                       296313
SAR                       181583
Military                  149443
Undefined                 105785
Law enforcement            62767
Reserved                   61489
Towing                     36496
Port tender                35561
Sailing                    20171
Diving                     17371
Spare 1                    13337
Pleasure                   11966
Anti-pollution    

In [45]:
# keep only Class A vessels
df = df[(df['Ship type'].isin(['Cargo', 'Tanker', 'Passenger']))].reset_index(drop=True)
print(f"Rows after filter (Ship types = Cargo, Tanker, Passenger): {len(df)}")
df.head()

Rows after filter (Ship types = Cargo, Tanker, Passenger): 5952931


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,ROT,SOG,COG,Width,Length,Ship type
0,29/03/2024 00:00:01,Class A,255806328,54.721793,12.368540,0.0,10.0,213.9,16.0,107.0,Cargo
1,29/03/2024 00:00:02,Class A,636021210,56.640483,7.389258,5.4,13.1,56.6,32.0,229.0,Cargo
2,29/03/2024 00:00:02,Class A,636021210,56.640483,7.389258,5.4,13.1,56.6,32.0,229.0,Cargo
3,29/03/2024 00:00:04,Class A,265410000,57.602262,11.566203,-1.1,15.5,257.3,33.0,183.0,Passenger
4,29/03/2024 00:00:04,Class A,265410000,57.602262,11.566203,-1.1,15.5,257.3,33.0,183.0,Passenger


In [46]:
# count datapoints per unique MMSI (uses existing `df`)
mmsi_counts = df['MMSI'].value_counts().sort_values(ascending=False)

# summary
print(f"Unique MMSI: {mmsi_counts.size}")
print(f"Total rows: {len(df)}")
print("\nTop 20 MMSI by number of datapoints:")
print(mmsi_counts.head(500))

# as a DataFrame for further analysis
mmsi_counts = mmsi_counts.rename_axis('MMSI').reset_index(name='count')
mmsi_counts.tail()

Unique MMSI: 825
Total rows: 5952931

Top 20 MMSI by number of datapoints:
MMSI
636023139    60168
255806368    45788
255806258    43964
636019884    41122
265859000    38549
             ...  
255805577     3533
209917000     3469
266436000     3404
220600000     3350
306157000     3325
Name: count, Length: 500, dtype: int64


,MMSI,count
820,266476000,6
821,246443000,3
822,538008274,3
823,215903000,2
824,211865680,1


In [47]:
# keep only MMSI with at least 1000 datapoints
rows_before = len(df)
unique_mmsi_before = df['MMSI'].nunique()

# compute counts (recompute to be safe)
counts = df['MMSI'].value_counts()
frequent_mmsi = counts[counts >= 1000].index

df = df[df['MMSI'].isin(frequent_mmsi)].reset_index(drop=True)

rows_after = len(df)
unique_mmsi_after = df['MMSI'].nunique()

# update mmsi_counts for further analysis
mmsi_counts = df['MMSI'].value_counts().rename_axis('MMSI').reset_index(name='count')

print(f"Rows before: 5952931, after filter: {rows_after} (removed {5952931 - rows_after})")
print(f"Unique MMSI before: {unique_mmsi_before}, after: {unique_mmsi_after}")
print("Top MMSI by datapoints (after filter):")
print(mmsi_counts.head(20))

Rows before: 5952931, after filter: 5889250 (removed 63681)
Unique MMSI before: 825, after: 658
Top MMSI by datapoints (after filter):
         MMSI  count
0   636023139  60168
1   255806368  45788
2   255806258  43964
3   636019884  41122
4   265859000  38549
5   255806493  37383
6   219592000  34557
7   211190000  34160
8   219016555  31458
9   538008157  31073
10  219019365  30619
11  257182000  29759
12  219000429  28464
13  255806305  28316
14  219019887  27712
15  219016938  27402
16  215459000  26998
17  266343000  26840
18  309998000  26742
19  511101231  26640


In [48]:
out_path = data_dir / "filtered_vessels.csv"
df.to_csv(out_path, index=False)
print(f"Saved {len(df)} rows to {out_path}")

Saved 5889250 rows to data/filtered_vessels.csv
